# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Feature Pipeline</span>

## 🗒️ This notebook is divided in 3 sections:
1. Parsing Data.
2. Preparing dataframes.
3. Feature Group Insertion.

## <span style='color:#ff5f27'> 📝 Imports

In [1]:
import pandas as pd
from datetime import datetime
import time 
import requests
import os 

from features import *

## <span style='color:#ff5f27'> 👮🏻‍♂️ API Keys

In [2]:
AIR_QUALITY_API_KEY = os.getenv('AIR_QUALITY_API_KEY')
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

date_today = datetime.now().strftime("%Y-%m-%d")

## <span style='color:#ff5f27'> 🕵🏻‍♂️ Parsing

In [3]:
cities = ['Kyiv','Stockholm','Sundsvall','Malmo']

data_air_quality = [get_air_quality_data(city,AIR_QUALITY_API_KEY) for city in cities]

data_weather = [get_weather_data(city,date_today,WEATHER_API_KEY) for city in cities]

## <span style='color:#ff5f27'> 🧑🏻‍🏫 Dataset Preparation

#### <span style='color:#ff5f27'> 👩🏻‍🔬 Air Quality Data

In [4]:
df_air_quality = get_air_quality_df(data_air_quality)

df_air_quality.head()

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,pm10_avg,pm10_max,pm10_min,pm25_avg,pm25_max,pm25_min,uvi_avg,uvi_max,uvi_min
0,Kyiv,2,1662930000000,99.9,1006.4,1,13.4,14,22,7,10,12,7,38,45,24,0,0,0
1,Stockholm,16,1662930000000,55.0,1014.0,16,18.0,22,27,19,6,7,5,15,18,11,0,0,0
2,Sundsvall,30,1662930000000,47.3,1010.8,20,18.1,21,29,15,5,6,3,11,13,8,0,0,0
3,Malmo,18,1662930000000,65.5,1013.3,5,20.0,19,27,10,10,12,8,29,35,23,0,0,0


#### <span style='color:#ff5f27'> 🌦 Weather Data

In [5]:
df_weather = get_weather_df(data_weather)

df_weather.head()

,city,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,Kyiv,1662930000000,13.9,11.3,12.5,13.9,11.3,12.5,10.7,88.9,...,24.1,10.1,256.9,1007.5,98.6,19.6,29.4,2.5,1.0,"Rain, Overcast"
1,Stockholm,1662930000000,18.9,6.0,12.3,18.9,6.0,12.2,8.6,80.9,...,25.9,11.2,133.9,1014.8,77.5,13.0,119.3,10.4,5.0,Partially cloudy
2,Sundsvall,1662930000000,16.3,4.9,10.3,16.3,2.4,9.5,7.0,80.4,...,25.9,12.6,248.1,1012.5,95.1,15.2,112.1,9.5,5.0,Overcast
3,Malmo,1662930000000,18.5,8.5,14.1,18.5,8.5,14.1,11.6,85.6,...,23.0,10.4,221.7,1015.7,71.7,14.1,168.8,14.7,7.0,Partially cloudy


## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [6]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/167
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">🪄 Retrieving Feature Groups</span>

In [7]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

## <span style="color:#ff5f27;">🧬 Inserting into Feature Groups</span>

In [8]:
air_quality_fg.insert(df_air_quality)

Uploading Dataframe: 0.00% |          | Rows 0/4 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/167/jobs/named/air_quality_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f7d0a4ced90>, None)

In [9]:
weather_fg.insert(df_weather)

Uploading Dataframe: 0.00% |          | Rows 0/4 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/167/jobs/named/weather_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f7d0a197df0>, None)

---